In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import re
import shutil
from pathlib import Path

from sam2util import (
    convert_mp4_to_images,
    convert_mp4_to_images_every_nth_frame,
    merge_videos,
)
from tqdm import tqdm

from model import dmd

In [ ]:
ROOT = Path.home() / 'source/driver-dataset'
image_dir = ROOT / 'images'
video_dir = ROOT / 'videos'
frame_sampling_frequency = 30


def list_dir(directory: Path) -> list[Path]:
    if not directory.is_dir():
        raise ValueError(f'`{directory}` is not a directory')
    return [dir for dir in directory.glob('*') if dir.is_dir()]

## Video to Images

In [ ]:
# Convert videos to images.
for path_to_process in tqdm(video_dir.rglob('*.mp4'), desc='Processing videos'):
    output_dir = str(path_to_process).replace('videos', 'images').replace('.mp4', '')
    convert_mp4_to_images_every_nth_frame(
        path_to_process, output_dir, n=frame_sampling_frequency
    )

In [ ]:
# Check the number of exported frames for each video.
frame_sum = 0
for path_to_process in video_dir.rglob('*.mp4'):
    output_dir_path = Path(
        str(path_to_process).replace('videos', 'images').replace('.mp4', '')
    )
    n = len(list(output_dir_path.glob('*.jpg')))
    frame_sum += n
    print(f'{path_to_process}: {n} frames')
print(f'Total frames: {frame_sum}')

In [ ]:
# Merge predicted videos into one video (test data)
# ~2 minutes

video_paths = list((ROOT / 'images').rglob('*.mp4'))
output_path = 'merged_video.mp4'
merge_videos(video_paths, output_path)

In [ ]:
# Check the number of images for each driver.
driver_dirs: list[Path] = list_dir(image_dir)
driver_dirs_and_subdirs: list[list[Path]] = [
    list_dir(person_dir) for person_dir in driver_dirs
]
all_dirs: list[Path] = [
    subdir for sublist in driver_dirs_and_subdirs for subdir in sublist
]
im_count = 0
for dir in all_dirs:
    images = list(dir.glob('*.jpg'))
    print(f'{dir.parent.name}/{dir.name}: {len(images)} images')
    im_count += len(images)
print(f'Total images: {im_count}')

## Train-Validation-Test Split

In [ ]:
def multiply_number_in_string(s: str, factor: int) -> str:
    """
    Example
    -------
    >>> multiply_number_in_string('frame_00002.jpg', 30)
    >>> 'frame_00060.jpg'
    """
    return re.sub(r'\d+', lambda x: f'{int(int(x.group()) * factor):05}', s)

In [ ]:
target_dataset = ROOT / Path('2025-03-23-driver-segmentation-dataset')
frame_sum = 0
for path_to_process in list(video_dir.rglob('*.mp4')) + list(
    list_dir(image_dir / 'dmd')
):
    output_dir_path = Path(
        str(path_to_process).replace('videos', 'images').replace('.mp4', '')
    )
    images = sorted(output_dir_path.glob('*.jpg'))

    # Check if there exists `*_fix` directory with updated masks (manual prompt corrections)
    path_to_fix = Path(str(output_dir_path) + '_fix')
    if path_to_fix.exists():
        masks = sorted((path_to_fix / 'sam2_output/masks').glob('*.png'))
    else:
        masks = sorted((output_dir_path / 'sam2_output/masks').glob('*.png'))

    assert len(images) == len(masks), 'Number of images and masks must be the same'

    n = len(images)

    val_size = 0.2
    val_n = int(n * val_size)
    train_n = n - 2 * val_n

    train_split = (images[:train_n], masks[:train_n])
    val_split = (images[train_n : train_n + val_n], masks[train_n : train_n + val_n])
    test_split = (images[train_n + val_n :], masks[train_n + val_n :])

    frame_dir = images[0].parent.parent.name + '_' + images[0].parent.name
    print(f'{frame_dir}: {n} frames')

    # move images and masks to train, val, test directories
    for split, split_name in zip(
        [train_split, val_split, test_split],
        ['train', 'validation', 'test'],
        strict=True,
    ):
        images, masks = split
        for image, mask in zip(images, masks):
            img_name = f'{frame_dir}-{image.name}'
            renamed_mask = multiply_number_in_string(
                mask.name, frame_sampling_frequency
            )
            mask_name = f'{frame_dir}-{renamed_mask}'

            image_dst = target_dataset / split_name / 'images' / img_name
            mask_dst = target_dataset / split_name / 'masks' / mask_name
            image_dst.parent.mkdir(parents=True, exist_ok=True)
            mask_dst.parent.mkdir(parents=True, exist_ok=True)

            # Copy images and masks to the destination directories
            shutil.copy(image, image_dst)
            shutil.copy(mask, mask_dst)

    frame_sum += n
print(f'Total frames: {frame_sum}')

In [ ]:
(
    len(list((target_dataset / 'train' / 'images').glob('*.jpg'))),
    len(list((target_dataset / 'validation' / 'images').glob('*.jpg'))),
    len(list((target_dataset / 'test' / 'images').glob('*.jpg'))),
)

## Example Video to Images

In [ ]:
for vid in ['anomal', 'normal']:
    convert_mp4_to_images(
        ROOT / f'videos/2021_08_31_geordi_enyaq/{vid}.mp4',
        ROOT / f'2024-10-28-driver-all-frames/2021_08_31_geordi_enyaq/{vid}',
    )

In [ ]:
dirs_to_convert = [
    '2021_11_18_dans_enyaq',
    '2021_09_06_poli_enyaq',
    '2021_11_05_michal_enyaq',
    '2021_11_18_jakubh_enyaq',
]

In [ ]:
for dir_name in dirs_to_convert:
    for vid in ['anomal', 'normal']:
        convert_mp4_to_images(
            video_dir / dir_name / f'{vid}.mp4',
            ROOT / '2024-10-28-driver-all-frames' / dir_name / vid / 'images',
        )

## DMD

Copy images from session clips to a folder.

In [ ]:
SOURCE_ROOT = Path.home() / 'source/driver-dataset/dmd'
TARGET_ROOT = Path.home() / 'source/driver-dataset/images'

In [ ]:
for session in dmd.get_all_sessions():
    subdirs = sorted([p for p in (SOURCE_ROOT / session).rglob('rgb') if p.is_dir()])
    all_images = sorted(
        [
            img
            for subdir in subdirs
            for img in subdir.glob('*.jpg')
            if int(img.stem) % frame_sampling_frequency == 0
        ],
        key=lambda p: p.stem,
    )
    output_dir = TARGET_ROOT / 'dmd' / session
    output_dir.mkdir(parents=True, exist_ok=True)

    for img in tqdm(all_images, desc=session):
        new_name = img.stem[1:] + img.suffix
        shutil.copy(img, output_dir / new_name)